In [ ]:
!pip install gradio transformers bitsandbytes accelerate peft trl pandas datasets

In [ ]:
!pip install transformers

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread
import pandas as pd
import bitsandbytes

In [ ]:
!pip install --upgrade pandas datasets transformers

import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

# Configuration
dataset_id = "jojogo9/Food_Recipes"
model_id = "eaglet/Testing"
output_model = "llm-007"
start_row = 600  # Starting row
end_row = 650  # Ending row
sample_size = 50  # Sample size for the dataset
max_length = 128  # Max length for tokenization

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# !pip install --upgrade pandas datasets transformers

# import torch
# from datasets import load_dataset, Dataset
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Configuration
# dataset_id = "jojogo9/Food_Recipes"
# model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# output_model = "llm-007"
# start_row = 150  # Starting row
# end_row = 250  # Ending row
# sample_size = 100  # Sample size for the dataset
# max_length = 128  # Max length for tokenization

# # Check device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Function to prepare training data with start and end row parameters
def prepare_train_data(data_id, start_row=0, end_row=None, sample_size=50):
    # Load the dataset
    data = load_dataset(data_id, split="train")
    data_df = data.to_pandas()

    # Select the rows between start_row and end_row
    if end_row is not None:
        data_df = data_df.iloc[start_row:end_row]
    else:
        data_df = data_df.iloc[start_row:]

    # If the resulting dataset is larger than sample_size, sample the data
    if len(data_df) > sample_size:
        data_df = data_df.sample(n=sample_size, random_state=42)

    # Format the text column as required
    data_df["text"] = data_df.apply(
        lambda x: (
            "im_start user\n" + x["ingredients"] + "\nim_end\n" +
            "im_start assistant\n" + x["name"] + "\nim_end\n" +
            "im_start steps: " + x["steps"] + "\nim_end\n"
        ),
        axis=1
    )

    # Convert the DataFrame back to a Dataset
    data = Dataset.from_pandas(data_df)

    return data

# Prepare the training data
data = prepare_train_data(dataset_id, start_row=start_row, end_row=end_row, sample_size=sample_size)

# Function to get model and tokenizer
def get_model_and_tokenizer(model_id):
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        # load_in_8bit=True  # If 4-bit isn't supported directly
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

# Load model and tokenizer
model, tokenizer = get_model_and_tokenizer(model_id)

# Function to tokenize, encode, and truncate the dataset
def encode_and_truncate_dataset(dataset, tokenizer, max_length=128):
    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            padding="max_length",  # Pad to the maximum length
            truncation=True,       # Truncate to the maximum length
            max_length=max_length  # Maximum length of sequences
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    return tokenized_dataset

# Encode and truncate the dataset
encoded_dataset = encode_and_truncate_dataset(data, tokenizer, max_length)

# Example to check the first encoded item
print(encoded_dataset[0])

# Save the dataset if needed
encoded_dataset.save_to_disk("encoded_dataset")

# Now your encoded_dataset is ready for training with the language model


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

# Define your LoraConfig
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Define a function to get model and tokenizer
def get_model_and_tokenizer(model_id):
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        # Remove quantization_config argument
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# Example usage
model_id = 'eaglet/Testing'
model, tokenizer = get_model_and_tokenizer(model_id)


In [ ]:
# Define the training arguments
training_arguments = TrainingArguments(
    output_dir="output_model",
    per_device_train_batch_size=2,  # Reduce batch size 8
    gradient_accumulation_steps=32,   # Increase accumulation steps 8
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=3,
    max_steps=50, #PUT 250 LATER
    fp16=True,
    # push_to_hub=True
)

# Assume 'data' is your training dataset
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024
)
trainer.train()

In [ ]:
model

In [ ]:
trainer.evaluate(encoded_dataset)

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch

import os

model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype=torch.float16,load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)
model_path = "/content/output_model/checkpoint-50"
# model_path = "/recipe-generator-G97"

peft_model = PeftModel.from_pretrained(model,model_path,from_transformers=True,device_map="auto")
model = peft_model.merge_and_unload()

In [ ]:
!pip install huggingface_hub --q
!huggingface-cli login

In [ ]:
# model.push_to_hub("eaglet/Testing")
# tokenizer.push_to_hub("eaglet/Testing")
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import HfApi, HfFolder, Repository

# Step 1: Load your fine-tuned model and tokenizer
model_name = "/content/output_model/checkpoint-50"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 2: Create a repository on Hugging Face
api = HfApi()
token = HfFolder.get_token()
repo_id = "eaglet/Testing"  # Replace with your details

# api.create_repo(repo_id=repo_id, token=token)

# Step 3: Clone the repository and push the model and tokenizer
repo = Repository(local_dir="/content/local", clone_from=repo_id)

# Save the model and tokenizer locally
model.save_pretrained("/content/local")
tokenizer.save_pretrained("/content/local")

# Push the model and tokenizer to the Hugging Face Hub
repo.push_to_hub(commit_message="Initial commit")


In [ ]:
# from huggingface_hub import HfApi, Repository

# # Replace with your Hugging Face username and model name
# username = "eaglet"
# model_name = "JulyTesting"

# # Initialize the API
# api = HfApi()

# # Clone the existing repository
# # api.upload_folder(
# #     repo_id=f"{username}/{model_name}",
# #     folder_path="/content/output_model/checkpoint-250",
# #     commit_message="Updated the fine-tuned model"
# # )
# api.upload_folder(
#     repo_id=f"{username}/{model_name}",
#     folder_path="/content/output_model/checkpoint-250",
#     path_in_repo="checkpoint-250"
# )

In [ ]:
def formated_prompt(quest):
  return f"<user>user:\n{quest}\n<assistant>assistant:\n"

In [ ]:
# from transformers import GenerationConfig
# from time import perf_counter

# def generate_resp(user_input):
#   prompt = formated_prompt(user_input)
#   inputs = tokenizer([prompt], return_tensors="pt")
#   generation_config = GenerationConfig(penalty_alpha=0.6,do_sample=True,
#                                          max_new_tokens=1024,pad_token_id=tokenizer.eos_token_id)
#   start_time = perf_counter()
#   inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
#   outputs = model.generate(**inputs,generation_config=generation_config)
#   # print(tokenizer.decode(outputs[0],skip_special_tokens=True))
#   output_time = perf_counter() - start_time
#   print("Time taken ",round(output_time,2))
#   return tokenizer.decode(outputs[0],skip_special_tokens=True)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from time import perf_counter

model_name = "/content/output_model/checkpoint-50"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def generate_resp(user_input):
    # Format the prompt
    prompt = formated_prompt(user_input)

    # Tokenize and move inputs to the correct device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Define the generation configuration
    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        max_new_tokens=2048,
        pad_token_id=tokenizer.eos_token_id
    )

    # Start timing
    start_time = perf_counter()

    # Generate the response
    with torch.no_grad():  # Disable gradient calculations for inference
        outputs = model.generate(**inputs, generation_config=generation_config)

    # Calculate the time taken
    output_time = perf_counter() - start_time
    print("Time taken", round(output_time, 2))

    # Decode and return the generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)




In [ ]:
# Example usage
print(generate_resp(user_input="give an output of 5 different recepies (with short name) and its respective ingredient list and also its respective steps to prepare the dish (point wise).also give the type of cuisine (eg asian, thai etc). the given ingreidents are chicken, spices, sauces, cheese, bread"))